In [1]:
# Import local files
from manage_path import *
from get_data import *
from compute_lda import *

/home/raymond/anaconda3/envs/topicModeling/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
data = load_pickle(file_name="FINRA_TRACE_2014.pkl.zip")

Getting data from/home/raymond/Research/FINRA_TRACE/Data/Pickle/FINRA_TRACE_2014.pkl.zip...
Data getting success from /home/raymond/Research/FINRA_TRACE/Data/Pickle/FINRA_TRACE_2014.pkl.zip!


In [3]:
data.dtypes

BOND_SYM_ID                    object
CUSIP_ID                       object
SCRTY_TYPE_CD                  object
ENTRD_VOL_QT                  float64
RPTD_PR                       float64
RPT_SIDE_CD                    object
Year                          float64
TRD_EXCTN_DTTM         datetime64[ns]
TRD_RPT_DTTM           datetime64[ns]
Report_Dealer_Index            object
Contra_Party_Index             object
TRC_ST                         object
RATING_MR                      object
RATING_MR_Numeric             float64
dtype: object

In [4]:
data['price'] = (data['ENTRD_VOL_QT'] * data['RPTD_PR'])/100

In [5]:
data['price'].describe()

count    6.323262e+06
mean     6.073740e+05
std      2.391643e+06
min      0.000000e+00
25%      1.103970e+04
50%      3.379530e+04
75%      2.457833e+05
max      1.154484e+09
Name: price, dtype: float64

In [6]:
"""Compute Dc_v4 which is count of bonds on given dealer and day seperated buy and sell"""
data['document_date'] = data['TRD_EXCTN_DTTM'].dt.date.apply(lambda x: str(x))
create_buy_document_no_source_vectorize = np.vectorize(create_buy_document_no_source)
create_sell_document_no_source_vectorize = np.vectorize(create_sell_document_no_source)
client_to_delete_vectorize = np.vectorize(client_to_delete)
print("creating documents ......")
# Add new column Dc_v4_S which is the string representation of report dealer buy on the specific day
data['Dc_v4_S'] = create_sell_document_no_source_vectorize(data['Report_Dealer_Index'].values,data['Contra_Party_Index'].values,data['document_date'].values)
# Add new column Dc_v4_B which is the string representation of report dealer sell on the specific day
data['Dc_v4_B'] = create_buy_document_no_source_vectorize(data['Report_Dealer_Index'].values,data['Contra_Party_Index'].values,data['document_date'].values)
print("documents created!!")

creating documents ......
documents created!!


In [7]:
type(data['Dc_v4_S'].iloc[7])

str

In [8]:
1 != 2

True

In [9]:
data_sub = data[['Dc_v4_S','Dc_v4_B','BOND_SYM_ID','price']].copy()
data_gb_sell = data_sub[data_sub['Dc_v4_S']!='nan'].groupby(by=['Dc_v4_S','BOND_SYM_ID'])
data_gb_buy = data_sub[data_sub['Dc_v4_B']!='nan'].groupby(by=['Dc_v4_B','BOND_SYM_ID'])

In [ ]:
#sell_matrix = data_gb_sell['price'].sum().unstack(fill_value=0)

In [13]:
data_gb_sell['price'].sum().to_sparse().unstack(level=-1)

KeyboardInterrupt: 

In [ ]:
sell_df = data_gb_sell['price'].sum().to_frame()

In [ ]:
sell_df.describe()

In [ ]:
sell_df['']

In [ ]:
bins=[0,10000,100000,1000000,10000000,sell_df[sell_df['price']>0]['price'].max()]
labels = [i for i in range(len(bins)-1)]
pd.cut(sell_df[sell_df['price']>0]['price'],bins=bins,labels=labels)

In [ ]:
pd.DataFrame(preprocessing.minmax_scale(sell_df[sell_df['price']>0],feature_range=(0, 5))).describe()

In [ ]:
sell_df.groupby(level=0).count().iloc[12::].describe()

In [ ]:
data_gb_sell['price'].sum().describe()

In [ ]:
from scipy.stats import mstats

In [ ]:
data['price_winsor'] = pd.Series(mstats.winsorize(data['price'].values,limits=[0.10,0.10]))

In [ ]:
pd.Series(preprocessing.minmax_scale(data['price_winsor'],feature_range=[0,5])).describe()

In [ ]:
data['price_winsor']